In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import re
from datetime import datetime, date
pd.set_option('display.max_columns', None)
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import hstack
from scipy.sparse import vstack
from scipy import sparse
from scipy.sparse import csr_matrix
from sklearn.preprocessing import OneHotEncoder
import pickle
from sklearn import metrics
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV # Import from model_selection
from sklearn.model_selection import RandomizedSearchCV # Import from model_selection
from sklearn.preprocessing import LabelBinarizer,LabelEncoder
from scipy.stats import randint as sp_randint
from sklearn import linear_model
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import pickle
import joblib

ModuleNotFoundError: No module named 'xgboost'

In [ ]:
# we'll be only using train, test data
from google.colab import files

# Upload file
uploaded = files.upload()

Saving TEST_FINAL.csv to TEST_FINAL.csv
Saving Train.csv to Train.csv


In [ ]:
train_df=pd.read_csv("Train.csv")
test_df=pd.read_csv("TEST_FINAL.csv")

In [ ]:
train_df = train_df.rename(columns={'Date': 'Sales_Date'})

In [ ]:
test_df = test_df.rename(columns={'Date': 'Sales_Date'})

In [ ]:
print(train_df.shape)

print(train_df.columns)

train_df.head()

(188340, 10)
Index(['ID', 'Store_id', 'Store_Type', 'Location_Type', 'Region_Code',
       'Sales_Date', 'Holiday', 'Discount', '#Order', 'Sales'],
      dtype='object')


,ID,Store_id,Store_Type,Location_Type,Region_Code,Sales_Date,Holiday,Discount,#Order,Sales
0,T1000001,1,S1,L3,R1,2018-01-01,1,Yes,9,7011.84
1,T1000002,253,S4,L2,R1,2018-01-01,1,Yes,60,51789.12
2,T1000003,252,S3,L2,R1,2018-01-01,1,Yes,42,36868.20
3,T1000004,251,S2,L3,R1,2018-01-01,1,Yes,23,19715.16
4,T1000005,250,S2,L3,R4,2018-01-01,1,Yes,62,45614.52


In [ ]:
test_df = test_df.rename(columns={'Date': 'Sales_Date'})

In [ ]:
print(test_df.shape)

print("\nColumns:",test_df.columns)

test_df.head()

(22265, 8)

Columns: Index(['ID', 'Store_id', 'Store_Type', 'Location_Type', 'Region_Code',
       'Sales_Date', 'Holiday', 'Discount'],
      dtype='object')


,ID,Store_id,Store_Type,Location_Type,Region_Code,Sales_Date,Holiday,Discount
0,T1188341,171,S4,L2,R3,2019-06-01,0,No
1,T1188342,172,S1,L1,R1,2019-06-01,0,No
2,T1188343,173,S4,L2,R1,2019-06-01,0,No
3,T1188344,174,S1,L1,R4,2019-06-01,0,No
4,T1188345,170,S1,L1,R2,2019-06-01,0,No


In [ ]:
#Checking null values
test_df.isnull().sum()/ len(test_df)

ID               0.0
Store_id         0.0
Store_Type       0.0
Location_Type    0.0
Region_Code      0.0
Sales_Date       0.0
Holiday          0.0
Discount         0.0
dtype: float64

In [ ]:
# Dropping rows where user id is missing
print(test_df.shape)

test_df = test_df.dropna(subset = ['ID'])

print(test_df.shape)

(22265, 8)
(22265, 8)


In [ ]:
test_df['ID'].nunique()

22265

In [ ]:
test_df_concat = test_df.groupby('ID', as_index=False).agg(lambda x: x.tolist())

print(test_df_concat.shape)

test_df_concat.head()

(22265, 8)


,ID,Store_id,Store_Type,Location_Type,Region_Code,Sales_Date,Holiday,Discount
0,T1188341,[171],[S4],[L2],[R3],[2019-06-01],[0],[No]
1,T1188342,[172],[S1],[L1],[R1],[2019-06-01],[0],[No]
2,T1188343,[173],[S4],[L2],[R1],[2019-06-01],[0],[No]
3,T1188344,[174],[S1],[L1],[R4],[2019-06-01],[0],[No]
4,T1188345,[170],[S1],[L1],[R2],[2019-06-01],[0],[No]


In [ ]:
# Function to convert list into strings

def list_to_str_store(Store_Type):

    """
    Function to convert list into strings

    parameters: Store_Type

    returns : Store_Type

    """
    action = [ str(i) for i in Store_Type ]

    action = [ re.sub('nan','',i) for i in Store_Type ]

    action = ','.join(Store_Type)

    return Store_Type

In [ ]:
test_df_concat['Store_Type'] = test_df_concat['Store_Type'].apply(list_to_str_store)

test_df_concat['Store_Type'].head()

0    [S4]
1    [S1]
2    [S4]
3    [S1]
4    [S1]
Name: Store_Type, dtype: object

In [ ]:
test_df_concat["Location_Type"] = test_df_concat["Location_Type"].apply(list_to_str_store)

test_df_concat["Location_Type"].head()

0    [L2]
1    [L1]
2    [L2]
3    [L1]
4    [L1]
Name: Location_Type, dtype: object

In [ ]:
# Function to convert list into strings

def list_to_str_Discount(Discount):

    """
    Function to convert list into strings

    parameters: Discount

    returns : Discount

    """

    Discount = [ str(i) for i in Discount ]

    Discount = [ re.sub('nan','',i) for i in Discount ]

    Discount = ','.join(set(Discount)) #keeping only unique devices that the user has used

    return Discount

In [ ]:
test_df_concat['Discount'] = test_df_concat['Discount'].apply(list_to_str_Discount)

test_df_concat['Discount'].head()

0    No
1    No
2    No
3    No
4    No
Name: Discount, dtype: object

In [ ]:
# correcting the datatypes
train_df['Sales_Date'] = pd.to_datetime(train_df['Sales_Date'])
train_df['#Order']= train_df['#Order'].astype('Int64')

In [ ]:
train_df['date_Sale_day'] = train_df.Sales_Date.dt.weekday

train_df['date_Sale_month'] = train_df.Sales_Date.dt.month

train_df['date_Sale_year'] = train_df.Sales_Date.dt.year


In [ ]:
# Convert 'Sales_Date' to datetime objects
test_df['Sales_Date'] = pd.to_datetime(test_df['Sales_Date'])

# Now you can extract weekday, month, and year
test_df['date_Sale_day'] = test_df.Sales_Date.dt.weekday
test_df['date_Sale_month'] = test_df.Sales_Date.dt.month
test_df['date_Sale_year'] = test_df.Sales_Date.dt.year

In [ ]:
def order_median(Order): # Add Order as an argument

    """
    Function to replace order outliers with median order

    parameters: order

    returns : order

"""

    if Order < 10 or Order > 100:

        return 20

    else:

        return Order # Return the Order value

In [ ]:
train_df['#Order'] = train_df['#Order'].fillna(-1)

train_df['#Order'] = train_df['#Order'].apply(order_median)

In [ ]:
#creating age buckets

bins = [i for i in range(15,106,5)]

def order_interv(Order):

    """

    Function takes age and returns interval.

    parameters:  order

    returns: order interval

    """

    for i in range(len(bins)):

        if Order < bins[i]:

            return i

In [ ]:
train_df['order_interv'] = train_df['#Order'].apply(lambda x: order_interv(x))

In [ ]:
# mode replacement for first_affiliate_tracked

train_df.Holiday.fillna('untracked',inplace = True)
test_df.Holiday.fillna('untracked',inplace = True)

In [ ]:
train_df.Location_Type.replace('-unknown-','OTHER', inplace=True)
test_df.Location_Type.replace('-unknown-','OTHER', inplace=True)

In [ ]:
#getting labels

y = train_df['Region_Code']

In [ ]:
#Checking null values
train_df.isnull().sum()

ID                 0
Store_id           0
Store_Type         0
Location_Type      0
Region_Code        0
Sales_Date         0
Holiday            0
Discount           0
#Order             0
Sales              0
date_Sale_day      0
date_Sale_month    0
date_Sale_year     0
order_interv       0
dtype: int64

In [ ]:
#Checking null values in test data
test_df.isnull().sum()

ID                 0
Store_id           0
Store_Type         0
Location_Type      0
Region_Code        0
Sales_Date         0
Holiday            0
Discount           0
date_Sale_day      0
date_Sale_month    0
date_Sale_year     0
dtype: int64

In [ ]:
print(train_df.shape,y.shape)
print(test_df.shape)

(188340, 14) (188340,)
(22265, 11)


In [ ]:
#Count vectorizer for action feature

# Create Count Vectorizer
count_vec_action = CountVectorizer(tokenizer=lambda x: x.split(','))

#building vocabulary using train data
count_vec_action.fit(train_df['Store_Type'].values)

#vectorization on train and test data
train_df_action_count_vec=count_vec_action.transform(train_df['Store_Type'].values)
test_df_action_count_vec=count_vec_action.transform(test_df['Store_Type'].values)

print("After vectorizations")
print(train_df_action_count_vec.shape)
print(test_df_action_count_vec.shape)

After vectorizations
(188340, 4)
(22265, 4)


In [ ]:
#looking at the features
action_feat=count_vec_action.get_feature_names_out()

action_feat

array(['s1', 's2', 's3', 's4'], dtype=object)

In [ ]:
train_df_action_count_vec_dense=train_df_action_count_vec.toarray() #converting to dense matrix

test_df_action_count_vec_dense=test_df_action_count_vec.toarray() #converting to dense matrix

train_df_action_count_vec_df = pd.DataFrame(train_df_action_count_vec_dense, columns=action_feat) #converting it to a df for merging

test_df_action_count_vec_df = pd.DataFrame(test_df_action_count_vec_dense, columns=action_feat) #converting it to a df for merging

print(train_df_action_count_vec_df.shape)

print(test_df_action_count_vec_df.shape)

(188340, 4)
(22265, 4)


In [ ]:
#Count vectorizer for action type feature

# Create Count Vectorizer
count_vec_action_type = CountVectorizer(tokenizer=lambda x: x.split(','))

#building vocabulary using train data
count_vec_action_type.fit(train_df['Location_Type'].values)

#vectorization on train and test data
train_df_action_type_count_vec=count_vec_action_type.transform(train_df['Location_Type'].values)
test_df_action_type_count_vec=count_vec_action_type.transform(test_df['Location_Type'].values)

print("After vectorizations")
print(train_df_action_type_count_vec.shape)
print(test_df_action_type_count_vec.shape)

After vectorizations
(188340, 5)
(22265, 5)


In [ ]:
#looking at the features
action_type_feat=count_vec_action_type.get_feature_names_out()

action_type_feat

array(['l1', 'l2', 'l3', 'l4', 'l5'], dtype=object)

In [ ]:
train_df_action_type_count_vec_dense=train_df_action_type_count_vec.toarray() #converting to dense matrix

test_df_action_type_count_vec_dense=test_df_action_type_count_vec.toarray() #converting to dense matrix

train_df_action_type_count_vec_df = pd.DataFrame(train_df_action_type_count_vec_dense, columns=action_type_feat) #converting it to a df for merging

test_df_action_type_count_vec_df = pd.DataFrame(test_df_action_type_count_vec_dense, columns=action_type_feat) #converting it to a df for merging

print(train_df_action_type_count_vec_df.shape)

print(test_df_action_type_count_vec_df.shape)

(188340, 5)
(22265, 5)


In [ ]:
#Count vectorizer for discount feature

# Create Count Vectorizer
count_vec_action_detail = CountVectorizer(tokenizer=lambda x: x.split(','))

#building vocabulary using train data
count_vec_action_detail.fit(train_df['Discount'].values)

#vectorization on train and test data
train_df_action_detail_count_vec=count_vec_action_detail.transform(train_df['Discount'].values)
test_df_action_detail_count_vec=count_vec_action_detail.transform(test_df['Discount'].values)

print("After vectorizations")
print(train_df_action_detail_count_vec.shape)
print(test_df_action_detail_count_vec.shape)

After vectorizations
(188340, 2)
(22265, 2)


In [ ]:
#looking at the features
action_detail_feat=count_vec_action_detail.get_feature_names_out()

action_detail_feat

array(['no', 'yes'], dtype=object)

In [ ]:
train_df_action_detail_count_vec_dense=train_df_action_detail_count_vec.toarray() #converting to dense matrix

test_df_action_detail_count_vec_dense=test_df_action_detail_count_vec.toarray() #converting to dense matrix

train_df_action_detail_count_vec_df = pd.DataFrame(train_df_action_detail_count_vec_dense, columns=action_detail_feat) #converting it to a df for merging

test_df_action_detail_count_vec_df = pd.DataFrame(test_df_action_detail_count_vec_dense, columns=action_detail_feat) #converting it to a df for merging

print(train_df_action_detail_count_vec_df.shape)

print(test_df_action_detail_count_vec_df.shape)

(188340, 2)
(22265, 2)


In [ ]:
train_df_final_df= pd.concat([train_df,train_df_action_count_vec_df, train_df_action_type_count_vec_df,train_df_action_detail_count_vec_df], axis=1)

test_df_final_df= pd.concat([test_df,test_df_action_count_vec_df, test_df_action_type_count_vec_df,test_df_action_detail_count_vec_df], axis=1)

In [ ]:
print(train_df_final_df.shape)
print(test_df_final_df.shape)

(188340, 25)
(22265, 22)


In [ ]:
# creating a list to store all the column names
col_lst = [i for i in train_df.columns]

col_lst

['ID',
 'Store_id',
 'Store_Type',
 'Location_Type',
 'Region_Code',
 'Sales_Date',
 'Holiday',
 'Discount',
 '#Order',
 'Sales',
 'date_Sale_day',
 'date_Sale_month',
 'date_Sale_year',
 'order_interv']

In [ ]:
col_lst.extend(action_feat) #extend the list by adding new elements into it

col_lst.extend(action_type_feat)

col_lst.extend(action_detail_feat)

len(col_lst)

25

In [ ]:
le = LabelEncoder() #making the true lables numeric for ML modelling

y = le.fit_transform(y)

y

array([0, 0, 0, ..., 1, 1, 0])

In [ ]:
y.shape

(188340,)

We'll need to save the test data, along with the label encoder to get the countries back from their labels along with that we'll be saving the best model.



In [ ]:
#saving train dataset
train_df_final_df.to_csv("final_train_dataset.csv", index=False)

#saving the test dataset
test_df_final_df.to_csv("final_test_dataset.csv", index=False)

#saving the label encoder
pickle.dump(le, open("label_encoder.pickle", "wb"))

In [ ]:
train_df_final_df.drop(['ID','Store_Type','Location_Type','Store_id','Region_Code','Sales_Date','Discount'],axis=1,inplace = True)
test_df_final_df.drop(['ID','Store_Type','Location_Type','Store_id','Region_Code','Sales_Date','Discount'],axis=1,inplace = True)

In [ ]:
# https://www.kaggle.com/davidgasquez/ndcg-scorer

def dcg_score(y_true, y_score, k=3):

    """Discounted cumulative gain (DCG) at rank K.

    Parameters
    ----------
    y_true : array, shape = [n_samples]
        Ground truth (true relevance labels).
    y_score : array, shape = [n_samples, n_classes]
        Predicted scores.
    k : int
        Rank.

    Returns
    -------
    score : float
    """

    order = np.argsort(y_score)[::-1] # sorts the indices of y_score in descending order. This helps us to rank the predictions from the highest to the lowest score.
    y_true = np.take(y_true, order[:k]) #reorders y_true based on the sorted indices and selects the top-k values. This aligns the true labels with the top-k predicted scores.

    gain = 2 ** y_true - 1 #calculate gain or numerator part of the dcg formula

    discounts = np.log2(np.arange(len(y_true)) + 2) #calculate discount or denominator part of the dcg formula
    return np.sum(gain / discounts)


def ndcg_score(ground_truth, predictions, k=5):

    """Normalized discounted cumulative gain (NDCG) at rank K.

    Normalized Discounted Cumulative Gain (NDCG) measures the performance of a
    recommendation system based on the graded relevance of the recommended
    entities. It varies from 0.0 to 1.0, with 1.0 representing the ideal
    ranking of the entities.

    Parameters
    ----------
    ground_truth : array, shape = [n_samples]
        Ground truth (true labels represended as integers).
    predictions : array, shape = [n_samples, n_classes]
        Predicted probabilities.
    k : int
        Rank.

    Returns
    -------
    score : float

    Example
    -------
    >>> ground_truth = [1, 0, 2]
    >>> predictions = [[0.15, 0.55, 0.2], [0.7, 0.2, 0.1], [0.06, 0.04, 0.9]]
    >>> score = ndcg_score(ground_truth, predictions, k=2)
    1.0
    >>> predictions = [[0.9, 0.5, 0.8], [0.7, 0.2, 0.1], [0.06, 0.04, 0.9]]
    >>> score = ndcg_score(ground_truth, predictions, k=2)
    0.6666666666
    """

    lb = LabelBinarizer() #Initializes a binarizer to convert multi-class labels to binary indicators.
    lb.fit(range(predictions.shape[1] + 1)) # predictions.shape[1] equals number of classes it'll be 12 in this case
    T = lb.transform(ground_truth) #it'll reshape each ground truth lable to number of classes that is 5 since we are predicting for 5 countries
    print(f'The original ground truth shape is{ground_truth.shape}')
    print(f'The transformed ground truth shape is{T.shape}')
    scores = []

    # Iterate over each y_true and compute the DCG score
    for y_true, y_score in zip(T, predictions):
        actual = dcg_score(y_true, y_score, k) #to compute the DCG score for the actual predictions.
        best = dcg_score(y_true, y_true, k) #to compute the DCG score for the ideal predictions (i.e., using the true labels as predictions).
        score = float(actual) / float(best)
        scores.append(score)

    return np.mean(scores) #Returns the mean of the NDCG scores for all samples.


# NDCG Scorer function
ndcg_scorer = make_scorer(ndcg_score, needs_proba=True, k=3)

## Testing with Logistic regression

In [ ]:
params =  {'C':[0.0001, 0.001, 0.01]}

lr = linear_model.LogisticRegression(multi_class="multinomial",solver="lbfgs")

clf = RandomizedSearchCV(lr, params, verbose=10, n_iter=3 ,n_jobs=-1,scoring=ndcg_scorer)

In [ ]:
clf.fit(train_df_final_df,y)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


RandomizedSearchCV(estimator=LogisticRegression(multi_class='multinomial'),
                   n_iter=3, n_jobs=-1,
                   param_distributions={'C': [0.0001, 0.001, 0.01]},
                   scoring=make_scorer(ndcg_score, needs_proba=True, k=3),
                   verbose=10)

In [ ]:
clf.best_params_

{'C': 0.01}

In [ ]:
pred_y = clf.predict_proba(train_df_final_df)

In [ ]:
pred_y.shape

(188340, 4)

If you see above the shape of pred_y is number_of_samples, num_of_classes here as we know that there are 12 unique destination countries our model gives probability scores for each of the countries since it's a multi-class classification task.

In [ ]:

s = ndcg_score(y, pred_y, k=5)

s

The original ground truth shape is(188340,)
The transformed ground truth shape is(188340, 5)


0.7030730072810685

## Testing with RF

In [ ]:
# training of the model takes time please do not run unless requried
params = {
'n_estimators' : [200, 700, 800, 1000, 1200],
'max_depth' : [15, 20, 25, 30, 35, 50],
'min_samples_split' : [2, 3, 5, 8],
'min_samples_leaf' : [1, 2, 5, 10] }

rf = RandomForestClassifier()

clf = RandomizedSearchCV(rf, params, verbose=10,n_jobs=2,scoring=ndcg_scorer)

In [ ]:
clf.fit(train_df_final_df,y) #takes more than 2 and half hour to run

Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(estimator=RandomForestClassifier(), n_jobs=2,
                   param_distributions={'max_depth': [15, 20, 25, 30, 35, 50],
                                        'min_samples_leaf': [1, 2, 5, 10],
                                        'min_samples_split': [2, 3, 5, 8],
                                        'n_estimators': [200, 700, 800, 1000,
                                                         1200]},
                   scoring=make_scorer(ndcg_score, needs_proba=True, k=3),
                   verbose=10)

In [ ]:
pred_y = clf.predict_proba(train_df_final_df)

In [ ]:
clf.best_params_

{'n_estimators': 200,
 'min_samples_split': 2,
 'min_samples_leaf': 10,
 'max_depth': 15}

In [ ]:
# Access the best estimator from the RandomizedSearchCV object
best_clf = clf.best_estimator_

In [ ]:
#Train ndcg score

s = ndcg_score(y, pred_y, k=5)

s

The original ground truth shape is(188340,)
The transformed ground truth shape is(188340, 5)


0.7925478845204602

Testing on test data

In [ ]:
test_id = test_df['ID'].values

In [ ]:
#saving ml model
pickle.dump(best_clf, open("best_ml_model.pickle", "wb"))

## Testing with XgBoost [Homework]

In [ ]:
param_grid = {
'max_depth': sp_randint(3, 20),
'learning_rate': [0.001, 0.01, 0.1, 0.2],
'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
'min_child_weight': [0.25,0.5, 1.0, 3.0, 5.0, 7.0],
'gamma': [0, 0.25, 0.3,0.35,0.45,0.5,0.6,0.8,1.0],
'reg_lambda': [0.1,0.2,0.4,0.5,0.6,0.8,1.0,10.0],
'n_estimators':[100,200,500,1000,2000],
'colsample_bytree':[0.1,0.3,0.5,1],
'colsample_bylevel':[0.1,0.3,0.5,1]
}

gb = xgb.XGBClassifier(objective='multi:softmax',eval_metric= 'mlogloss')

clf = RandomizedSearchCV(gb, param_grid,n_jobs=-1,verbose=10,scoring=ndcg_scorer, random_state=42)

NameError: name 'xgb' is not defined

In [ ]:
clf.fit(train_df_final_df,y)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


RandomizedSearchCV(estimator=LogisticRegression(multi_class='multinomial'),
                   n_iter=3, n_jobs=-1,
                   param_distributions={'C': [0.0001, 0.001, 0.01]},
                   scoring=make_scorer(ndcg_score, needs_proba=True, k=3),
                   verbose=10)

In [ ]:
pred_y = clf.predict_proba(train_df_final_df)

In [ ]:
# Train ndcg score

s = ndcg_score(y, pred_y, k=5)

s

The original ground truth shape is(188340,)
The transformed ground truth shape is(188340, 5)


0.7030730072810685

In [ ]:
test_id = test_df['ID'].values

In [ ]:
pip install streamlit

Could not fetch URL https://pypi.org/simple/streamlit/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/streamlit/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate in certificate chain (_ssl.c:1002)'))) - skipping
Could not fetch URL https://pypi.org/simple/pip/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/pip/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate in certificate chain (_ssl.c:1002)'))) - skipping
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement streamlit (from versions: none)
ERROR: No matching distribution found for streamlit
